In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_PROJECT"] = "xmen"

In [2]:
import sys
sys.path.append('../scripts')

In [3]:
import openai
from pathlib import Path
import pandas as pd
from tqdm.auto import tqdm

from transformers import AutoModelForSeq2SeqLM,Text2TextGenerationPipeline
from evaluation import error_analysis, get_scores
from dataset import load_data, get_dataloader
from generative.transformers_util import get_training_args, get_tokenizer

In [4]:
import hydra
from hydra import compose, initialize

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(config_path=Path('..'), job_name='foo', version_base='1.1')
config = compose(config_name='experiment.yaml')

In [5]:
openai.api_key = config.api_key

In [6]:
BATCH_SIZE = 32
SPLIT = 'test'

# Load Data

In [7]:
training_args = get_training_args(config, report_to="none")
tokenizer = get_tokenizer(config)

base_path = Path('..')
train_df, val_df, test_df = load_data(base_path / config.data.cnf_tsv_path, base_path / config.data.controls_tsv_path)
train_dataset, val_dataset, test_dataset = get_dataloader(train_df, val_df, test_df, tokenizer)

/home/Florian.Borchert/miniconda3/envs/ellipses/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [8]:
if SPLIT == 'valid':
    df = val_df
elif SPLIT == 'test':
    df = test_df

In [14]:
output_path = Path(f'results_{SPLIT}')
output_path.mkdir(exist_ok=True)

In [15]:
resolutions = list(val_df.full_resolution)
samples = list(val_df.raw_sentence)

# Initialize Model

In [16]:
model = AutoModelForSeq2SeqLM.from_pretrained("../data/ellipses/model")

# TOP 1 generation from seq-to-seq model

In [18]:
%%time
pipeline = Text2TextGenerationPipeline(model=model, tokenizer=tokenizer, max_length=config.generation_max_length, device=0, batch_size=BATCH_SIZE)

predictions = pipeline(samples)

errors = error_analysis([prediction["generated_text"] for prediction in predictions], resolutions, samples)

CPU times: user 58.4 s, sys: 832 ms, total: 59.3 s
Wall time: 59.3 s


In [19]:
errors.error_type.value_counts() / len(errors)

tn         0.491257
tp         0.415487
replace    0.018318
fp         0.017485
delete     0.016653
fn         0.016653
insert     0.014155
complex    0.009992
Name: error_type, dtype: float64

In [38]:
errors.to_parquet(output_path / 'results_top1.parquet')

In [22]:
%%time
get_scores(errors, SPLIT)

CPU times: user 1min 19s, sys: 122 ms, total: 1min 19s
Wall time: 1min 21s


{'test/exact_match': 0.906744379683597,
 'test/gleu': 0.9863425249277339,
 'test/edit_distance_rel': 0.9496346247979276}

# Best out of TOP k predictions from seq-to-seq model

In [23]:
from evaluation import relative_edit_distance

def evaluate_top_k(model, tokenizer, data, beams, generation_max_length=config.generation_max_length, debug=False):
    pipeline = Text2TextGenerationPipeline(
        model=model, tokenizer=tokenizer, max_length=generation_max_length, num_beams=beams, num_return_sequences=beams, device=0, batch_size=BATCH_SIZE)

    originals = list(data.raw_sentence)
    resolutions = list(data.full_resolution)
    outputs = pipeline(originals)

    predictions = []
    for i, resolution in enumerate(resolutions):
        generations = [entry['generated_text'] for entry in outputs[i]]
        scores = [relative_edit_distance(gen, resolution, originals[i]) for gen in generations]
        if debug and max(scores) == 1 and scores.index(max(scores)) != 0:
            print(generations)
            print(scores.index(max(scores)))
            print(resolution)
        predictions.append(generations[scores.index(max(scores))])

    errors = error_analysis(predictions, resolutions, list(data.raw_sentence))

    return errors, outputs

In [102]:
%%time
k = 2
errors_k, outputs_k = evaluate_top_k(model, tokenizer, val_df, k)

CPU times: user 4min 22s, sys: 227 ms, total: 4min 22s
Wall time: 4min 22s


In [105]:
errors_k.error_type.value_counts() / len(errors_k)

tn         0.500416
tp         0.435470
replace    0.014988
insert     0.013322
delete     0.012490
fn         0.008326
fp         0.008326
complex    0.006661
Name: error_type, dtype: float64

In [106]:
results_k = pd.concat([errors_k, pd.Series(outputs_k, name='outputs_k')], axis=1)

In [107]:
results_k.to_parquet(output_path / f'results_top{k}.parquet')

In [ ]:
%%time
get_scores(results_k, SPLIT)

In [29]:
%%time
get_scores(results_k, SPLIT)

CPU times: user 1min 17s, sys: 151 ms, total: 1min 17s
Wall time: 1min 19s


{'test/exact_match': 0.9458784346378019,
 'test/gleu': 0.99178128963028,
 'test/edit_distance_rel': 0.978016404376749}

# Using ChatGPT/GPT to determine the best fit of the top k options

## Prompts

In [30]:
def generate_multiple_choice_prompt(original, predictions):
    # Das folgende Problem wurde von Yann LeCun gestellt, der sehr an der Kompetenz von Künstlicher Intelligenz, wie dir, zweifelt: 
    beginning = "Dir werden im Folgenden ein Satz gezeigt, welcher sogennannte Koordinationsellipsen enthält. Das Ziel ist es diese zu aufzulösen. Ein Beispiel wäre 'Ibrutinib, ein Inhibitor der Bruton-Tyrosinkinase (BTK), ist in Deutschland als Erstlinien- und Rezidivtherapiee in der CLL zugelassen.' Die richtige Auflösung wäre 'Ibrutinib, ein Inhibitor der Bruton-Tyrosinkinase (BTK), ist in Deutschland als Erstlinientherapie und Rezidivtherapiee in der CLL zugelassen.' Dir werden zu den Beispielen Antwortmöglichkeiten gegeben und du sollst dann entscheiden, welche dieser Optionen die Koordinationsellipsen korrekt auflöst.\n\n"
    original = f"Der originale Satz: '{original}'\n\n"
    answers = "Deine Antwortmöglichkeiten:\n" + "".join(f"{i+1}) '{prediction}'\n" for i, prediction in enumerate(predictions))
    end = "\nWelche Antwort ist die richtige? Antworte nur mit der Zahl und keiner Erklärung"

    return beginning + original + answers + end

In [31]:
def generate_prompt_true_false(original, predictions):
    beginning = "Sie haben ein Modell entwickelt, das Koordinationsellipsen in Sätzen erkennt und auflöst. Das Modell gibt fünf verschiedene Versionen des ursprünglichen Satzes zurück, wobei die erste Version die wahrscheinlichste ist. Bitte lesen Sie sich die erste Version des Satzes sorgfältig durch und entscheiden Sie, ob diese Version korrekt ist und den ursprünglichen Satz mit aufgelösten Koordinationsellipsen wiedergibt. Bitte antworten Sie mir nur 'Ja' oder 'Nein' und keiner Erklärung!\n\n"
    original = f"Ursprünglicher Satz: '{original}'\n\n"
    answer = f"Erste Version: '{predictions[0]}'\n\n"

    return beginning + original + answer

In [76]:
def generate_prompt_other_options(original, predictions):
    beginning = "Sie haben entschieden, dass die erste Version des Satzes, die vom Modell als die wahrscheinlichste ausgewählt wurde, nicht korrekt ist und den ursprünglichen Satz mit aufgelösten Koordinationsellipsen nicht vollständig wiedergibt. Das Modell gibt vier weitere Versionen des Satzes zurück, die als die nächst wahrscheinlichsten Versionen ausgewählt wurden. Bitte lesen Sie sich diese vier Versionen sorgfältig durch und wählen Sie die Version aus, die Ihrer Meinung nach am besten den ursprünglichen Satz mit aufgelösten Koordinationsellipsen wiedergibt.\n\n"
    answers = "" + "".join(f"{i+1} - '{prediction}'\n" for i, prediction in enumerate(predictions[1:]))
    end = "\nBitte antworten Sie nur mit der richtigen Nummer und ohne Erklärung."
    return beginning + answers + end

In [33]:
def generate_best_fit_prompts(samples, outputs, ground_truths, prompt_fn):
    prompt_df = []
    for i, sample in enumerate(samples):
        generations = [entry['generated_text'] for entry in outputs[i]]
        ground_truth = ground_truths[i]
        true_index = generations.index(ground_truth) if ground_truth in generations else -1
        
        prompt_df.append({
            'input' : sample,
            'generations' : generations,
            'true_index' : true_index,
            'prompt' : prompt_fn(sample, generations)
        })
    
    return pd.DataFrame(prompt_df)

## OpenAI API calls

In [34]:
from retry import retry
import logging
logging.basicConfig()

In [35]:
@retry(tries=3, delay=2)
def get_openai_response_chatgpt(messages):
    return openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0301",
        messages=[
            {"role": role, "content": text} for (role, text) in messages
        ],
        temperature=0,
        max_tokens=100,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )["choices"][0]["message"]["content"]

In [36]:
@retry(tries=3, delay=2)
def get_openai_response_gpt3(prompt):
    return openai.Completion.create(
        model="text-davinci-003",
        prompt= prompt,
        temperature=0,
        max_tokens=100,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )["choices"][0]["text"]

## Query GPT-3.5 to pick 1 from k options

In [39]:
results_5 = pd.read_parquet(output_path / f'results_top5.parquet')

In [40]:
from tqdm.auto import tqdm
import re

def call_api_best_fit(prompt_df, skip_if_1st_unchanged=True, debug=False):
    predictions = []

    for i, row in tqdm(list(prompt_df.iterrows())):
        generations = row.generations
        sample = row.input
        prompt = row.prompt
        status = 'success'
        
        if skip_if_1st_unchanged and generations[0] == sample:
            if debug:
                print(f'{i}) answer: {0}')
                print('--------------------------------')
            predictions.append({'status': 'skipped', 'prediction' : generations[0], 'index' : 0, 'answer' : None})

        else:
            message = [("user", prompt)]
            answer = get_openai_response_chatgpt(message)

            numbers = re.findall(r'\d+', answer)
            if len(numbers) > 1:
                if debug:
                    print(f'more numbers than expected {numbers}')
                status = 'error_multiple'
                index = numbers[0]
            if len(numbers) == 0:
                if debug:
                    print(f'no numbers found')
                index = 0
                status = 'error_no_numbers'
            else:
                index = int(numbers[0]) - 1
                if index >= len(generations):
                    if debug:
                        print(f'Index is out of bounds. Something went wrong with the API Answer. Defaulting to 0')
                    index = 0
                    status = 'error_out_of_bounds'

            if debug:
                print(f'{i}) answer: {index}')
                print('--------------------------------')
            predictions.append({'status' : status, 'prediction' : generations[index], 'index' : index, 'answer' : answer})
    
    return pd.DataFrame(predictions)

In [41]:
originals_k = list(results_5.original)
resolutions = list(results_5.ground_truth)

In [44]:
# Prepare prompts
prompts_best_fit = generate_best_fit_prompts(originals_k, outputs_k, resolutions, generate_multiple_choice_prompt)

In [45]:
%%time
#Call OpenAI API
predictions = call_api_best_fit(prompts_best_fit)

  0%|          | 0/1201 [00:00<?, ?it/s]

<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>cloudflare</center>
</body>
</html>
), retrying in 2 seconds...
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>api.openai.com | 502: Bad gateway</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">

    

    <div id="cf-error-details" class="p-0">
        <header class="mx-auto pt-1

CPU times: user 3.29 s, sys: 628 ms, total: 3.92 s
Wall time: 31min 28s


In [46]:
prompts2preds_bestfit = pd.concat([prompts_best_fit, predictions], axis=1)

In [47]:
prompts2preds_bestfit.to_parquet(output_path / 'prompts2preds_bestfit.parquet')

In [48]:
errors_best_fit = error_analysis(prompts2preds_bestfit.prediction, resolutions, samples)

In [49]:
errors_best_fit.error_type.value_counts() / len(errors_best_fit)

tn         0.502082
tp         0.223980
insert     0.079933
delete     0.061615
replace    0.055787
fn         0.053289
complex    0.016653
fp         0.006661
Name: error_type, dtype: float64

In [50]:
%%time
get_scores(errors_best_fit, SPLIT)

CPU times: user 1min 19s, sys: 60.2 ms, total: 1min 19s
Wall time: 1min 20s


{'test/exact_match': 0.7260616153205662,
 'test/gleu': 0.9711623527883968,
 'test/edit_distance_rel': 0.9037117897230146}

## Two-Step Approach: 1. Ask if 1st option is correct 2. choose from options 2-k otherwise

In [52]:
results_5 = pd.read_parquet(output_path / 'results_top5.parquet')
results_5.head()

,pred,ground_truth,original,error_type,outputs_k
0,Die Inzidenz synchroner Zweittumoren oder von ...,Die Inzidenz synchroner Zweittumoren oder von ...,Die Inzidenz synchroner Zweittumoren oder von ...,tp,[{'generated_text': 'Die Inzidenz synchroner Z...
1,In einer Studie des Instituts für Qualität und...,In einer Studie des Instituts für Qualität und...,In einer Studie des Instituts für Qualität und...,tp,[{'generated_text': 'In einer Studie des Insti...
2,Bei Vorliegen einer cN1 Subgruppe oder N1 Subg...,Bei Vorliegen einer cN1 Subgruppe oder N1 Subg...,Bei Vorliegen einer cN1 oder N1 Subgruppe kann...,tp,[{'generated_text': 'Bei Vorliegen einer cN1 S...
3,Eine Radiochemotherapie soll nur an Einrichtun...,Eine Radiochemotherapie soll nur an Einrichtun...,Eine Radiochemotherapie soll nur an Einrichtun...,tp,[{'generated_text': 'Eine Radiochemotherapie s...
4,Ähnlich wie die Schleimhaut kann auch die äuße...,Ähnlich wie die Schleimhaut kann auch die äuße...,Ähnlich wie die Schleimhaut kann auch die äuße...,tp,[{'generated_text': 'Ähnlich wie die Schleimha...


In [57]:
top1_classifier_prompts = generate_best_fit_prompts(results_5.original, results_5.outputs_k, results_5.ground_truth, generate_prompt_true_false)
print(top1_classifier_prompts.iloc[0].prompt)

Sie haben ein Modell entwickelt, das Koordinationsellipsen in Sätzen erkennt und auflöst. Das Modell gibt fünf verschiedene Versionen des ursprünglichen Satzes zurück, wobei die erste Version die wahrscheinlichste ist. Bitte lesen Sie sich die erste Version des Satzes sorgfältig durch und entscheiden Sie, ob diese Version korrekt ist und den ursprünglichen Satz mit aufgelösten Koordinationsellipsen wiedergibt. Bitte antworten Sie mir nur 'Ja' oder 'Nein' und keiner Erklärung!

Ursprünglicher Satz: 'Die Inzidenz synchroner Zweittumoren oder von Fernmetastasen liegt bei Karzinomen der Mundhöhle, abhängig von der Größe des Primärtumors, zwischen 4% und 33%, wobei die Stadien T3/T4 und Patienten mit Lymphknotenbefall in Level IV besonders häufig betroffen sind [REF], [REF], [REF].'

Erste Version: 'Die Inzidenz synchroner Zweittumoren oder von Fernmetastasen liegt bei Karzinomen der Mundhöhle, abhängig von der Größe des Primärtumors, zwischen 4% und 33%, wobei die Stadien T3 und Stadien T4

In [58]:
from tqdm.auto import tqdm
import re

def call_api_top1_classifier(prompt_df, skip_if_1st_unchanged=True, debug=False):
    predictions = []
    for i, row in tqdm(list(prompt_df.iterrows())):
        sample = row.input
        generations = row.generations
        prompt = row.prompt

        if skip_if_1st_unchanged and generations[0] == sample:
            if debug:
                print(f'{i}) answer: {0}')
                print('--------------------------------')
            predictions.append({'status': 'skipped', 'prediction1' : generations[0], 'accept' : True, 'answer' : None})
        else:
            message = [("user", prompt)]
            answer = get_openai_response_chatgpt(message)
            extracted_answer = re.findall(r'Ja|Nein', answer)
            if len(extracted_answer) > 1:
                if debug:
                    print(f'more answers than expected: {answer}. Defaulting to 0')
                    print(f'{i}) answer: {0}')
                    print('--------------------------------')
                predictions.append({'status': 'error_multiple', 'prediction1' : generations[0], 'accept' : True, 'answer' : answer})
            elif len(extracted_answer) == 0:
                if debug:
                    print(f'No answer provided: {answer}. Defaulting to 0')
                    print(f'{i}) answer: {0}')
                    print('--------------------------------')
                predictions.append({'status': 'error_no_answer', 'prediction1' : generations[0], 'accept' : True, 'answer' : answer})
            elif extracted_answer[0] == 'Ja':
                if debug:
                    print(f'{i}) answer: {0}')
                    print('--------------------------------')
                predictions.append({'status': 'success', 'prediction1' : generations[0], 'accept' : True, 'answer' : answer})
            elif extracted_answer[0] == 'Nein':
                if debug:
                    print(f'{i}) answer: != 0')
                    print('--------------------------------')
                predictions.append({'status': 'success', 'prediction1' : None, 'accept' : False, 'answer' : answer})
    return pd.DataFrame(predictions)

In [59]:
predictions_1 = call_api_top1_classifier(top1_classifier_prompts, debug=False)

  0%|          | 0/1201 [00:00<?, ?it/s]

<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>api.openai.com | 502: Bad gateway</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">

    

    <div id="cf-error-details" class="p-0">
        <header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">
            <h1 class="inline-block sm:block sm:mb-2 font-light text-60 lg:text-4xl text-black-dark leading-tight mr-2">
       

In [60]:
prompt2predictions_1 = pd.concat([top1_classifier_prompts, predictions_1], axis=1)

In [61]:
prompt2predictions_1.to_parquet(output_path / 'prompt2predictions_1.parquet')

In [62]:
prompt2predictions_1.accept.value_counts()

True     1175
False      26
Name: accept, dtype: int64

In [66]:
accepted = prompt2predictions_1[prompt2predictions_1.accept]
len(accepted[accepted.true_index > 0]) / len(accepted)

0.03574468085106383

In [69]:
not_accepted = prompt2predictions_1[~prompt2predictions_1.accept]
len(not_accepted[not_accepted.true_index > 0]) / len(not_accepted)

0.2692307692307692

# Ask GPT to choose from remaining options

In [70]:
results_5 = pd.read_parquet(output_path / 'results_top5.parquet')
results_5.head()

,pred,ground_truth,original,error_type,outputs_k
0,Die Inzidenz synchroner Zweittumoren oder von ...,Die Inzidenz synchroner Zweittumoren oder von ...,Die Inzidenz synchroner Zweittumoren oder von ...,tp,[{'generated_text': 'Die Inzidenz synchroner Z...
1,In einer Studie des Instituts für Qualität und...,In einer Studie des Instituts für Qualität und...,In einer Studie des Instituts für Qualität und...,tp,[{'generated_text': 'In einer Studie des Insti...
2,Bei Vorliegen einer cN1 Subgruppe oder N1 Subg...,Bei Vorliegen einer cN1 Subgruppe oder N1 Subg...,Bei Vorliegen einer cN1 oder N1 Subgruppe kann...,tp,[{'generated_text': 'Bei Vorliegen einer cN1 S...
3,Eine Radiochemotherapie soll nur an Einrichtun...,Eine Radiochemotherapie soll nur an Einrichtun...,Eine Radiochemotherapie soll nur an Einrichtun...,tp,[{'generated_text': 'Eine Radiochemotherapie s...
4,Ähnlich wie die Schleimhaut kann auch die äuße...,Ähnlich wie die Schleimhaut kann auch die äuße...,Ähnlich wie die Schleimhaut kann auch die äuße...,tp,[{'generated_text': 'Ähnlich wie die Schleimha...


In [71]:
prompt2predictions_1 = pd.read_parquet(output_path / 'prompt2predictions_1.parquet')
prompt2predictions_1.head()

,input,generations,true_index,prompt,status,prediction1,accept,answer
0,Die Inzidenz synchroner Zweittumoren oder von ...,[Die Inzidenz synchroner Zweittumoren oder von...,1,"Sie haben ein Modell entwickelt, das Koordinat...",success,None,False,Nein.
1,In einer Studie des Instituts für Qualität und...,[In einer Studie des Instituts für Qualität un...,0,"Sie haben ein Modell entwickelt, das Koordinat...",success,In einer Studie des Instituts für Qualität und...,True,Ja.
2,Bei Vorliegen einer cN1 oder N1 Subgruppe kann...,[Bei Vorliegen einer cN1 Subgruppe oder N1 Sub...,0,"Sie haben ein Modell entwickelt, das Koordinat...",success,Bei Vorliegen einer cN1 Subgruppe oder N1 Subg...,True,Ja.
3,Eine Radiochemotherapie soll nur an Einrichtun...,[Eine Radiochemotherapie soll nur an Einrichtu...,0,"Sie haben ein Modell entwickelt, das Koordinat...",success,Eine Radiochemotherapie soll nur an Einrichtun...,True,Ja.
4,Ähnlich wie die Schleimhaut kann auch die äuße...,[Ähnlich wie die Schleimhaut kann auch die äuß...,0,"Sie haben ein Modell entwickelt, das Koordinat...",success,Ähnlich wie die Schleimhaut kann auch die äuße...,True,Ja.


In [77]:
other_options_prompts = generate_best_fit_prompts(results_5.original, results_5.outputs_k, results_5.ground_truth, generate_prompt_other_options)
print(other_options_prompts.iloc[0].prompt)

Sie haben entschieden, dass die erste Version des Satzes, die vom Modell als die wahrscheinlichste ausgewählt wurde, nicht korrekt ist und den ursprünglichen Satz mit aufgelösten Koordinationsellipsen nicht vollständig wiedergibt. Das Modell gibt vier weitere Versionen des Satzes zurück, die als die nächst wahrscheinlichsten Versionen ausgewählt wurden. Bitte lesen Sie sich diese vier Versionen sorgfältig durch und wählen Sie die Version aus, die Ihrer Meinung nach am besten den ursprünglichen Satz mit aufgelösten Koordinationsellipsen wiedergibt.

1 - 'Die Inzidenz synchroner Zweittumoren oder von Fernmetastasen liegt bei Karzinomen der Mundhöhle, abhängig von der Größe des Primärtumors, zwischen 4% und 33%, wobei die Stadien T3/Stadien T4 und Patienten mit Lymphknotenbefall in Level IV besonders häufig betroffen sind [REF], [REF], [REF].'
2 - 'Die Inzidenz synchroner Zweittumoren oder von Fernmetastasen liegt bei Karzinomen der Mundhöhle, abhängig von der Größe des Primärtumors, zwis

In [78]:
prompt2predictions_2 = pd.concat([prompt2predictions_1, other_options_prompts.prompt.rename('prompt2')], axis=1)
prompt2predictions_2.head()

,input,generations,true_index,prompt,status,prediction1,accept,answer,prompt2
0,Die Inzidenz synchroner Zweittumoren oder von ...,[Die Inzidenz synchroner Zweittumoren oder von...,1,"Sie haben ein Modell entwickelt, das Koordinat...",success,None,False,Nein.,"Sie haben entschieden, dass die erste Version ..."
1,In einer Studie des Instituts für Qualität und...,[In einer Studie des Instituts für Qualität un...,0,"Sie haben ein Modell entwickelt, das Koordinat...",success,In einer Studie des Instituts für Qualität und...,True,Ja.,"Sie haben entschieden, dass die erste Version ..."
2,Bei Vorliegen einer cN1 oder N1 Subgruppe kann...,[Bei Vorliegen einer cN1 Subgruppe oder N1 Sub...,0,"Sie haben ein Modell entwickelt, das Koordinat...",success,Bei Vorliegen einer cN1 Subgruppe oder N1 Subg...,True,Ja.,"Sie haben entschieden, dass die erste Version ..."
3,Eine Radiochemotherapie soll nur an Einrichtun...,[Eine Radiochemotherapie soll nur an Einrichtu...,0,"Sie haben ein Modell entwickelt, das Koordinat...",success,Eine Radiochemotherapie soll nur an Einrichtun...,True,Ja.,"Sie haben entschieden, dass die erste Version ..."
4,Ähnlich wie die Schleimhaut kann auch die äuße...,[Ähnlich wie die Schleimhaut kann auch die äuß...,0,"Sie haben ein Modell entwickelt, das Koordinat...",success,Ähnlich wie die Schleimhaut kann auch die äuße...,True,Ja.,"Sie haben entschieden, dass die erste Version ..."


In [92]:
def call_api_otheroptions_classifier(prompt_df, debug=False):
    predictions = []
    for i, row in tqdm(list(prompt_df.iterrows())):
        if row.accept:
            if debug:
                print(f'{i} was accepted previously.')
            predictions.append({'status2': 'accept_1', 'prediction2' : row.prediction1, 'answer2' : None})
        else:
            prompt1 = row.prompt
            answer = row.answer
            prompt2 = row.prompt2

            messages = [("user", prompt1), ("assistant", answer), ("user", prompt2)]

            answer2 = get_openai_response_chatgpt(messages)

            numbers = re.findall(r'\d+', answer2)
            if len(numbers) > 1:
                if debug:
                    print(f'more numbers than expected {numbers}')
                predictions.append({'status2': 'error_multiple', 'prediction2' : row.generations[1], 'answer2' : answer2})
            if len(numbers) == 0:
                if debug:
                    print(f'no numbers found')
                predictions.append({'status2': 'no_numbers', 'prediction2' : row.generations[1], 'answer2' : answer2})
            else:
                index = int(numbers[0])
                if index > 4:
                    if debug:
                        print(f'Index is out of bounds. Something went wrong with the API Answer. Defaulting to 0')
                    predictions.append({'status2': 'no_numbers', 'prediction2' : row.generations[1], 'answer2' : answer2})
                else:
                    predictions.append({'status2': 'success', 'prediction2' : row.generations[index], 'answer2' : answer2})
    return pd.DataFrame(predictions)

In [94]:
predictions_2 = call_api_otheroptions_classifier(prompt2predictions_2)

  0%|          | 0/1201 [00:00<?, ?it/s]

<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>api.openai.com | 502: Bad gateway</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">

    

    <div id="cf-error-details" class="p-0">
        <header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">
            <h1 class="inline-block sm:block sm:mb-2 font-light text-60 lg:text-4xl text-black-dark leading-tight mr-2">
       

In [96]:
result2 = pd.concat([prompt2predictions_2, predictions_2], axis=1)

In [97]:
errors_2 = error_analysis(result2.prediction2, val_df.full_resolution, result2.input)

In [99]:
errors_2.error_type.value_counts()# / len(errors_2)

tn         593
tp         487
replace     24
delete      24
insert      23
fn          21
fp          18
complex     11
Name: error_type, dtype: int64

In [89]:
%%time
get_scores(errors_2, SPLIT)

CPU times: user 1min 18s, sys: 75.6 ms, total: 1min 19s
Wall time: 1min 21s


{'eval/exact_match': 0.8984179850124896,
 'eval/gleu': 0.9864061103437068,
 'eval/edit_distance_rel': 0.9502187577476662}

In [87]:
# Baseline TOP 1 only
errors_top1 = pd.read_parquet(output_path / 'results_top1.parquet')

In [88]:
errors_top1.error_type.value_counts()

tn         590
tp         499
replace     22
fp          21
delete      20
fn          20
insert      17
complex     12
Name: error_type, dtype: int64

In [90]:
%%time
get_scores(errors_top1, SPLIT)

CPU times: user 1min 18s, sys: 83.9 ms, total: 1min 18s
Wall time: 1min 20s


{'test/exact_match': 0.906744379683597,
 'test/gleu': 0.9863425249277339,
 'test/edit_distance_rel': 0.9496346247979276}

In [91]:
errors_temp = error_analysis(result2.generations.map(lambda l: l[0]), val_df.full_resolution, result2.input)
errors_temp.error_type.value_counts()

tn         590
tp         497
delete      24
replace     22
fp          21
fn          20
insert      16
complex     11
Name: error_type, dtype: int64